In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import warnings
warnings.filterwarnings("ignore")

In [2]:
df_train = pickle.load(open("data_pickle/train_data.pkl",'rb'))

In [3]:
df_train

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs,112108,train,4,4,10,9.0
1,1,49683,4,0,Cucumber Kirby,83,4,fresh vegetables,produce,112108,train,4,4,10,9.0
2,1,13176,6,0,Bag of Organic Bananas,24,4,fresh fruits,produce,112108,train,4,4,10,9.0
3,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods,112108,train,4,4,10,9.0
4,1,10246,3,0,Organic Celery Hearts,83,4,fresh vegetables,produce,112108,train,4,4,10,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33818974,3421063,49235,1,1,Organic Half & Half,53,16,cream,dairy eggs,169679,train,30,0,10,4.0
33818975,3421063,14233,3,1,Natural Artesian Water,115,7,water seltzer sparkling water,beverages,169679,train,30,0,10,4.0
33819017,3421070,16953,2,1,Creamy Peanut Butter,88,13,spreads,pantry,139822,train,15,6,10,8.0
33819018,3421070,35951,1,1,Organic Unsweetened Almond Milk,91,16,soy lactosefree,dairy eggs,139822,train,15,6,10,8.0


In [19]:
%%time
product_features = pickle.load(open("new_features/product_features.pkl","rb"))
user_features = pickle.load(open("new_features/user_features.pkl","rb"))
user_features = user_features.drop(["reordered"],axis=1)
user_product_features = pickle.load(open("new_features/user_product_features.pkl","rb"))
# user_features = pd.read_csv("user_features_v6.csv")
# user_product_features = pd.read_csv("user_product_features_prior_data_v6.csv")

Wall time: 917 ms


In [20]:
%%time
#merge all product ,user, user_product features with prior_last_orders

merged_df = pd.merge(user_product_features, user_features,  how='outer', left_on=['user_id'], right_on = ['user_id'])
merged_df = pd.merge(merged_df, product_features,  how='outer', left_on=['product_id'], right_on = ['product_id'])

Wall time: 10.7 s


In [21]:
merged_df.tail(5)

,user_id,product_id,user_product_order_rate,user_product_reorder_rate,user_product_avg_position,user_product_orders_since_last,reorder_rate,unique_products,user_total_products,avg_cart_size,user_avg_days_between_orders,user_unique_products,user_reordered_products,user_reordered_products_ratio,product_reorder_rate,avg_pos_incart,aisle_id,department_id,aisle_reorder_rate,dept_reorder_rate
13307948,200463,8682,0.026316,0.000000,11.000000,2,0.184211,31,38,9.500000,22.500000,31,4.0,0.129032,0.000000,9.000000,114,17,0.288537,0.402178
13307949,203389,8682,0.005025,0.000000,7.000000,1,0.427136,114,199,11.705882,4.705882,114,0.0,0.000000,0.000000,9.000000,114,17,0.288537,0.402178
13307950,202557,43553,0.038690,0.923077,3.538462,13,0.607143,132,336,10.838710,8.741935,132,14.0,0.106061,0.923077,3.538462,64,7,0.649607,0.653460
13307951,203436,42338,0.052632,0.500000,10.000000,1,0.236842,29,38,12.666667,15.000000,29,0.0,0.000000,0.500000,10.000000,50,19,0.592415,0.574180
13307952,205420,28818,0.008475,0.000000,8.000000,15,0.228814,91,118,7.375000,14.812500,91,0.0,0.000000,0.000000,8.000000,130,14,0.499501,0.560922


In [22]:
%%time
upd_train_orders = df_train[['user_id','order_id','product_id','reordered']]
last_orders = upd_train_orders.groupby(['user_id'])['order_id'].max().reset_index(name = 'new_order_id')
order_details = df_train[['order_id','order_dow','order_hour_of_day','days_since_prior_order']]
order_details = order_details.drop_duplicates()

Wall time: 432 ms


In [23]:
order_details.head(3)

,order_id,order_dow,order_hour_of_day,days_since_prior_order
0,1,4,10,9.0
333,36,6,18,30.0
344,38,6,16,24.0


In [24]:
%%time
#merge latest orders (prior_last_orders) with above user and product feat

train_df = merged_df.merge(upd_train_orders,  how='left', left_on=['user_id','product_id'], right_on = ['user_id','product_id'])
train_df = train_df.merge( last_orders, on = 'user_id')
train_df.drop("order_id", axis = 1, inplace = True)
train_df.rename(columns = {'new_order_id':'order_id'}, inplace = True) 
train_df = train_df.merge( order_details, on = 'order_id')
train_df[['reordered']]= train_df[['reordered']].fillna(value=0.0)

Wall time: 25.6 s


In [25]:
train_df.columns

Index(['user_id', 'product_id', 'user_product_order_rate',
       'user_product_reorder_rate', 'user_product_avg_position',
       'user_product_orders_since_last', 'reorder_rate', 'unique_products',
       'user_total_products', 'avg_cart_size', 'user_avg_days_between_orders',
       'user_unique_products', 'user_reordered_products',
       'user_reordered_products_ratio', 'product_reorder_rate',
       'avg_pos_incart', 'aisle_id', 'department_id', 'aisle_reorder_rate',
       'dept_reorder_rate', 'reordered', 'order_id', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order'],
      dtype='object')

In [26]:
%%time
hour_reorder_rate = pickle.load(open("new_features/hour_reorder_rate.pkl","rb"))
product_days_since_prior_order_reorder_rate = pickle.load(open("new_features/product_days_since_prior_order_reorder_rate.pkl","rb"))
day_reorder_rate = pickle.load(open("new_features/day_reorder_rate.pkl","rb"))
user_days_since_prior_order_reorder_rate = pickle.load(open("new_features/user_days_since_prior_order_reorder_rate.pkl","rb"))
days_since_prior_reorder_rate = pickle.load(open("new_features/days_since_prior_reorder_rate.pkl","rb"))

Wall time: 1.24 s


In [33]:
days_since_prior_reorder_rate.head()

,user_id,product_id,days_since_prior_order,days_since_prior_reorder_rate
0,1,196,0.0,0.2
1,1,196,14.0,0.1
2,1,196,15.0,0.1
3,1,196,19.0,0.1
4,1,196,20.0,0.1


In [27]:
train_df = train_df.merge( hour_reorder_rate, on=['product_id','order_hour_of_day'], how = 'left')

In [28]:
train_df = train_df.merge( day_reorder_rate, on=['product_id','order_dow'], how = 'left')

In [30]:
train_df = train_df.merge( product_days_since_prior_order_reorder_rate, on=['product_id','days_since_prior_order'], how = 'left')

In [32]:
train_df = train_df.merge(user_days_since_prior_order_reorder_rate, on=['user_id','days_since_prior_order'], how = 'left')

In [34]:
train_df = train_df.merge( days_since_prior_reorder_rate, on=['product_id','user_id','days_since_prior_order'], how = 'left')

In [35]:
train_df.isna().sum()

user_id                                              0
product_id                                           0
user_product_order_rate                              0
user_product_reorder_rate                            0
user_product_avg_position                            0
user_product_orders_since_last                       0
reorder_rate                                         0
unique_products                                      0
user_total_products                                  0
avg_cart_size                                        0
user_avg_days_between_orders                         0
user_unique_products                                 0
user_reordered_products                              0
user_reordered_products_ratio                        0
product_reorder_rate                                 0
avg_pos_incart                                       0
aisle_id                                             0
department_id                                        0
aisle_reor

In [36]:
train_df = train_df.fillna(0)

In [37]:
train_df.columns

Index(['user_id', 'product_id', 'user_product_order_rate',
       'user_product_reorder_rate', 'user_product_avg_position',
       'user_product_orders_since_last', 'reorder_rate', 'unique_products',
       'user_total_products', 'avg_cart_size', 'user_avg_days_between_orders',
       'user_unique_products', 'user_reordered_products',
       'user_reordered_products_ratio', 'product_reorder_rate',
       'avg_pos_incart', 'aisle_id', 'department_id', 'aisle_reorder_rate',
       'dept_reorder_rate', 'reordered', 'order_id', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order', 'hour_reorder_rate',
       'day_reorder_rate', 'product_days_since_prior_order_reorder_rate',
       'user_days_since_prior_order_reorder_rate',
       'days_since_prior_reorder_rate'],
      dtype='object')

In [38]:
%%time
pickle.dump(train_df,open("data_model/train_orders_merged_df.pkl","wb"))

Wall time: 3.52 s


In [2]:
train_df = pickle.load(open("data_model/train_orders_merged_df.pkl",'rb'))

In [6]:
%%time
user_features_f15 = pickle.load(open("new_features/user_features_f15.pkl",'rb'))
train_df  = train_df.merge(user_features_f15,on="user_id",how="left")

Wall time: 16.1 s


In [7]:
%%time
user_features_l15 = pickle.load(open("new_features/user_features_l15.pkl",'rb'))
train_df  = train_df.merge(user_features_l15,on="user_id",how="left")

Wall time: 6.26 s


In [10]:
%%time
user_product_features_f15 = pickle.load(open("new_features/user_product_features_f15.pkl",'rb'))
train_df  = train_df.merge(user_product_features_f15,on=["user_id","product_id"],how="left")

Wall time: 12.8 s


In [11]:
%%time
user_product_features_l15 = pickle.load(open("new_features/user_product_features_l15.pkl",'rb'))
train_df  = train_df.merge(user_product_features_l15,on=["user_id","product_id"],how="left")

Wall time: 11.3 s


In [13]:
%%time
embedings = pickle.load(open("new_features/products_pca.pkl",'rb'))
train_df  = train_df.merge(embedings,on=["product_id"],how="left")

Wall time: 10.4 s


In [14]:
train_df.columns

Index(['user_id', 'product_id', 'user_product_order_rate',
       'user_product_reorder_rate', 'user_product_avg_position',
       'user_product_orders_since_last', 'reorder_rate', 'unique_products',
       'user_total_products', 'avg_cart_size', 'user_avg_days_between_orders',
       'user_unique_products_x', 'user_reordered_products',
       'user_reordered_products_ratio', 'product_reorder_rate',
       'avg_pos_incart', 'aisle_id', 'department_id', 'aisle_reorder_rate',
       'dept_reorder_rate', 'reordered', 'order_id', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order', 'hour_reorder_rate',
       'day_reorder_rate', 'product_days_since_prior_order_reorder_rate',
       'user_days_since_prior_order_reorder_rate',
       'days_since_prior_reorder_rate', 'reorder_rate_f15',
       'unique_products_f15', 'user_total_products_f15', 'avg_cart_size_f15',
       'user_avg_days_between_orders_f15', 'user_reordered_products_f15',
       'user_reordered_products_ratio_f15',

In [15]:
train_df = train_df.drop("user_unique_products_y",axis=1)
train_df = train_df.rename(columns={"user_unique_products_x":"user_unique_products"})

In [16]:
train_df.columns

Index(['user_id', 'product_id', 'user_product_order_rate',
       'user_product_reorder_rate', 'user_product_avg_position',
       'user_product_orders_since_last', 'reorder_rate', 'unique_products',
       'user_total_products', 'avg_cart_size', 'user_avg_days_between_orders',
       'user_unique_products', 'user_reordered_products',
       'user_reordered_products_ratio', 'product_reorder_rate',
       'avg_pos_incart', 'aisle_id', 'department_id', 'aisle_reorder_rate',
       'dept_reorder_rate', 'reordered', 'order_id', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order', 'hour_reorder_rate',
       'day_reorder_rate', 'product_days_since_prior_order_reorder_rate',
       'user_days_since_prior_order_reorder_rate',
       'days_since_prior_reorder_rate', 'reorder_rate_f15',
       'unique_products_f15', 'user_total_products_f15', 'avg_cart_size_f15',
       'user_avg_days_between_orders_f15', 'user_reordered_products_f15',
       'user_reordered_products_ratio_f15', '

In [19]:
train_df.isna().any()

user_id                      False
product_id                   False
user_product_order_rate      False
user_product_reorder_rate    False
user_product_avg_position    False
                             ...  
pca25                        False
pca26                        False
pca27                        False
pca28                        False
pca29                        False
Length: 77, dtype: bool

In [20]:
train_df = train_df.fillna(0)

In [21]:
%%time
pickle.dump(train_df,open("data_model/train_orders_merged_df_new.pkl","wb"))

Wall time: 38.1 s


In [39]:
%%time
#load all data files
aisles = pd.read_csv('data\\aisles.csv')
departments = pd.read_csv('data\\departments.csv')
prior_order = pd.read_csv('data\\order_products__prior.csv')
train_order = pd.read_csv('data\\order_products__train.csv')
orders = pd.read_csv('data\\orders.csv')
products = pd.read_csv('data\\products.csv')

#imputation
orders.fillna(value = 0, inplace = True)

Wall time: 10.1 s


In [40]:
%%time
#get test order details

order_details_test = orders[orders['eval_set'] == 'test']
order_details_test = order_details_test.drop(['eval_set'], axis = 1)

Wall time: 257 ms


In [41]:
%%time
# merge on user_product features, to get features on this order

test_df = pd.merge(order_details_test, user_product_features, on = ['user_id'], how = 'outer')
test_df.dropna(inplace =True)
test_df = test_df.merge(user_features, on = ['user_id'])
test_df = test_df.merge(product_features, on = ['product_id'])

Wall time: 10.4 s


In [42]:
%%time
test_df = test_df.merge( hour_reorder_rate, on=['product_id','order_hour_of_day'], how = 'left')
test_df = test_df.merge( day_reorder_rate, on=['product_id','order_dow'], how = 'left')
test_df = test_df.merge( product_days_since_prior_order_reorder_rate, on=['product_id','days_since_prior_order'], how = 'left')

test_df = test_df.merge( user_days_since_prior_order_reorder_rate, on=['user_id','days_since_prior_order'], how = 'left')
test_df = test_df.merge( days_since_prior_reorder_rate, on=['product_id','user_id','days_since_prior_order'], how = 'left')

Wall time: 27.8 s


In [43]:
test_df.isna().sum()

order_id                                             0
user_id                                              0
order_number                                         0
order_dow                                            0
order_hour_of_day                                    0
days_since_prior_order                               0
product_id                                           0
user_product_order_rate                              0
user_product_reorder_rate                            0
user_product_avg_position                            0
user_product_orders_since_last                       0
reorder_rate                                         0
unique_products                                      0
user_total_products                                  0
avg_cart_size                                        0
user_avg_days_between_orders                         0
user_unique_products                                 0
user_reordered_products                              0
user_reord

In [45]:
test_df = test_df.fillna(0)

In [46]:
%%time
pickle.dump(test_df,open("data_model/test_orders_merge_df.pkl","wb"))

Wall time: 2.01 s


In [22]:
%%time
test_df = pickle.load(open("data_model/test_orders_merge_df.pkl",'rb'))

Wall time: 2.04 s


In [23]:
%%time
user_features_f15 = pickle.load(open("new_features/user_features_f15.pkl",'rb'))
test_df  = test_df.merge(user_features_f15,on="user_id",how="left")

Wall time: 11.1 s


In [24]:
%%time
user_features_l15 = pickle.load(open("new_features/user_features_l15.pkl",'rb'))
test_df  = test_df.merge(user_features_l15,on="user_id",how="left")

Wall time: 4.52 s


In [25]:
%%time
user_product_features_f15 = pickle.load(open("new_features/user_product_features_f15.pkl",'rb'))
test_df  = test_df.merge(user_product_features_f15,on=["user_id","product_id"],how="left")

Wall time: 10.7 s


In [26]:
%%time
user_product_features_l15 = pickle.load(open("new_features/user_product_features_l15.pkl",'rb'))
test_df  = test_df.merge(user_product_features_l15,on=["user_id","product_id"],how="left")

Wall time: 8.06 s


In [27]:
%%time
embedings = pickle.load(open("new_features/products_pca.pkl",'rb'))
test_df  = test_df.merge(embedings,on=["product_id"],how="left")

Wall time: 4.23 s


In [28]:
test_df = test_df.drop("user_unique_products_y",axis=1)
test_df = test_df.rename(columns={"user_unique_products_x":"user_unique_products"})

In [29]:
test_df.isna().any()

order_id             False
user_id              False
order_number         False
order_dow            False
order_hour_of_day    False
                     ...  
pca25                False
pca26                False
pca27                False
pca28                False
pca29                False
Length: 77, dtype: bool

In [30]:
test_df = test_df.fillna(0)

In [31]:
%%time
pickle.dump(test_df,open("data_model/test_orders_merge_df_new.pkl","wb"))

Wall time: 37.8 s
